In [49]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [46]:
url = 'https://www.whed.net/results_institutions.php'


response = requests.get(url,headers=headers)

page_contents = response.text

soup = BeautifulSoup(page_contents, 'html.parser')

country = []

for i in range(1, 269):
    country.append(soup.find('select').find_all('option')[i].text)

In [47]:
len(country)

268

In [127]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def fetch_all(country):
    institutes = []
    cities = []
    urls = []

    driver = webdriver.Chrome()

    # Navigate to the website
    driver.get("https://www.whed.net/results_institutions.php")
    time.sleep(1)
    drop_down = Select(driver.find_element(By.XPATH, '//select'))
    drop_down.select_by_visible_text(country)

    all_institute = driver.find_element(By.XPATH, "//input[@id='membre2']")
    if not all_institute.is_selected():
        all_institute.click()

    button = driver.find_element(By.XPATH, "//input[@type='button']")
    button.click()
    time.sleep(1)
    
    try:
        results_per_page = Select(driver.find_element(By.XPATH, "//select[@name='nbr_ref_pge']"))
        results_per_page.select_by_visible_text('100')
    except:
        data = {'institute': institutes, 'city': cities, 'url': urls}
        return data


    # Get the total number of results
    try: 
        total_results_text = driver.find_element(By.XPATH, "//p[@class='infos']").text
        total_results = int(total_results_text.split()[0])
    except:
        total_results = 0

    max_iter = total_results // 100 + 1
    iterations = 0
    go_on = True

    while go_on:
        iterations += 1
        if total_results == 0:
            break
        time.sleep(1)
        
        try: 
            institutions = driver.find_elements(By.XPATH, "//li[contains(@class, 'clearfix plus')]")

            for institute in institutions:
                time.sleep(1)
                link = institute.find_element(By.XPATH, ".//h3/a")
                link.click()

                time.sleep(1)

                driver.switch_to.frame(driver.find_element(By.CSS_SELECTOR, ".fancybox-iframe"))

                try:
                    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//div[@class="detail_right"]/div[1]')))
                    time.sleep(1)
                    university = driver.find_element(By.XPATH, '//div[@class="detail_right"]/div[1]').text
                except:
                    university = 'Not-Found'

                try:
                    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//span[@class="libelle" and text()="City:"]/following-sibling::span[@class="contenu"]')))
                    time.sleep(1)
                    university_address = driver.find_element(By.XPATH, '//span[@class="libelle" and text()="City:"]/following-sibling::span[@class="contenu"]').text
                except:
                    university_address = 'Not-found'

                try:
                    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//span[@class="libelle" and text()="WWW:"]/following-sibling::span[@class="contenu"]/a')))
                    time.sleep(1)
                    university_webpage = driver.find_element(By.XPATH, '//span[@class="libelle" and text()="WWW:"]/following-sibling::span[@class="contenu"]/a').text
                except:
                    university_webpage = 'Webpage-Not-Found'

                institutes.append(university)
                cities.append(university_address)
                urls.append(university_webpage)

                driver.switch_to.parent_frame()
                time.sleep(2)
                close_button = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "/html/body/div[2]/div/div/a")))
                close_button.click()
                driver.switch_to.default_content()
                time.sleep(1)

            if iterations >= max_iter:
                go_on = False
                break

            time.sleep(2)
            next_page = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//a[@title='Next page']")))
            next_page.click()
            
        except Exception as e:
            print(e)

    # Close the WebDriver
    driver.quit()

    data = {'institute': institutes, 'city': cities, 'url': urls}
    return data
